# Application for the display of raster data from geoserver
## WMS initialisation

In [17]:
from owslib.wms import WebMapService
url = 'http://localhost:8082/geoserver/ows'
wms = WebMapService(url, version='1.1.1')
[i for i in wms.contents.keys() if i.startswith("geonode:")]

['geonode:NDVIs',
 'geonode:timeseries_overall',
 'geonode:DNI_yearly',
 'geonode:NDVI_20010101',
 'geonode:NDVI_20050101',
 'geonode:NDVI_20050301',
 'geonode:NDVI_20050601',
 'geonode:NDVI_20050901',
 'geonode:NDVI_20100101',
 'geonode:PVOUT',
 'geonode:PVOUT_yearly',
 'geonode:TEMP_yearly',
 'geonode:demand',
 'geonode:kwh_sq',
 'geonode:ne_10m_admin_0_countries',
 'geonode:timeseries']

In [18]:
layer = 'geonode:PVOUT'
wms[layer].title

'PVOUT'

In [19]:
wms[layer].styles

{'raster': {'title': 'Default Raster',
  'legend': 'http://localhost:8082/geoserver/wms?request=GetLegendGraphic&format=image%2Fpng&width=20&height=20&layer=geonode%3APVOUT'},
 'geonode:3color_gradient_PVOUT': {'title': 'SLD Cook Book: 3 color gradient',
  'legend': 'http://localhost:8082/geoserver/wms?request=GetLegendGraphic&format=image%2Fpng&width=20&height=20&layer=geonode%3APVOUT&style=3color_gradient_PVOUT'}}

## Scrolling raster time scroller preparation

In [20]:
layer = "geonode:PVOUT"
name = wms[layer].title
print(wms[layer].timepositions)
print(wms[layer].boundingBoxWGS84)
print(wms[layer].styles)

['2018-01-01T00:00:00.000Z', '2018-02-01T00:00:00.000Z', '2018-03-01T00:00:00.000Z', '2018-04-01T00:00:00.000Z', '2018-05-01T00:00:00.000Z', '2018-06-01T00:00:00.000Z', '2018-07-01T00:00:00.000Z', '2018-08-01T00:00:00.000Z', '2018-09-01T00:00:00.000Z', '2018-10-01T00:00:00.000Z', '2018-11-01T00:00:00.000Z', '2018-12-01T00:00:00.000Z']
(19.0, 34.0, 30.0, 42.0)
{'raster': {'title': 'Default Raster', 'legend': 'http://localhost:8082/geoserver/wms?request=GetLegendGraphic&format=image%2Fpng&width=20&height=20&layer=geonode%3APVOUT'}, 'geonode:3color_gradient_PVOUT': {'title': 'SLD Cook Book: 3 color gradient', 'legend': 'http://localhost:8082/geoserver/wms?request=GetLegendGraphic&format=image%2Fpng&width=20&height=20&layer=geonode%3APVOUT&style=3color_gradient_PVOUT'}}


In [21]:
lon = (wms[layer].boundingBox[0] + wms[layer].boundingBox[2]) / 2.
lat = (wms[layer].boundingBox[1] + wms[layer].boundingBox[3]) / 2.
center = lat, lon

time_interval = '{0}/{1}'.format(
    wms[layer].timepositions[0].strip(),
    wms[layer].timepositions[-1].strip()
)
style = 'geonode:3color_gradient_PVOUT'

if style not in wms[layer].styles:
    style = None

## Application: Time scrolling raster application

In [22]:
import folium
from folium import plugins
import os

#from branca.element import Figure
#width, height = 430, 280
#fig = Figure(width=width, height=height)

print("This map consists of one time scrolling layer: {}".format(wms["geonode:PVOUT"].title))
print("Abstract:")
print(wms["geonode:PVOUT"].abstract)
#print(catalog[catalog["layer_name"]=="PVOUT"].abstract.values[0])

lon, lat = 24, 38

m = folium.Map(
    tiles = None,
    location=[lat, lon],
    zoom_start=6,
    control_scale=True
)

folium.raster_layers.WmsTileLayer(
    url=url,
    layers='geonode:ne_10m_admin_0_countries',
    name='countries',
    fmt='image/png',
    overlay=False,
    control=True,
).add_to(m)

pvout_monthly = folium.raster_layers.WmsTileLayer(
    url=url,
    name="PVOUT_monthly",
    styles=None,
    fmt='image/png8',
    transparent=True,
    layers="geonode:PVOUT",
    overlay=True
).add_to(m)

time = plugins.TimestampedWmsTileLayers(
    pvout_monthly,
    period='P1M',
    time_interval=time_interval
).add_to(m)

m.save(os.path.join('results', 'PVOUT_time_scroller.html'))

m

This map consists of one time scrolling layer: PVOUT
Abstract:
Longterm monthly average of potential photovoltaic electricity production
(PVOUT) in kWh/kWp, calculated for all months and covering the years from 1994
to 2018


We observe that during the summer period we have lighter colors which means higher PV production potential obviously...